In [180]:
import json
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, shape

In [181]:
with open('data/1645657200.json', 'r') as f:
    ex = json.load(f)

In [182]:
def get_coords(points):
    shapes = []
    if type(points[0]) is dict:
        coords = []
        for p in points:
            coords.append([p.get('lat'), p.get('lng')])
        shapes.append(coords)
        return shapes
    for p in points:
        shapes.append(np.column_stack([p[1::2], p[::2]]))
    return shapes

shape_dicts = []

for field_id, field_dict in ex['fields'].items():
    c = get_coords(field_dict['points'])

    if len(c) == 0:
        # Simple polygon
        polygon = Polygon(c[0])
    else:
        # Multiple polygons, to be combined
        polygon = MultiPolygon(Polygon(x) for x in c)
    
    shape_dict = dict(id=field_id, name=field_dict['name'], geometry=polygon)
    
    shape_dicts.append(shape_dict)

gdf = gpd.GeoDataFrame(shape_dicts)
    
#df = pd.DataFrame(c[0])

#geom = [shape(i['geometry']) for i in occupied]
#gdf = gpd.GeoDataFrame({'geometry': geom}, crs='EPSG:4326')
# https://stackoverflow.com/questions/38961816/geopandas-set-crs-on-points
#gdf = gdf.to_crs('EPSG:3857')  # convert to metric projection
# https://gis.stackexchange.com/questions/254413/how-to-fix-hole-lies-outside-shell
# https://gis.stackexchange.com/questions/253224/geopandas-buffer-using-geodataframe-while-maintaining-the-dataframe
#gdf['geometry'] = gdf.geometry.buffer(0)
#joined = gdf.dissolve(by=None)
#return float(round(joined.area, 2) / 1e6)"

In [272]:
ua_territory = gpd.GeoDataFrame(pd.concat([gdf[0:8], gdf[10:11]]))

In [273]:
from ipyleaflet import Map, GeoData

In [274]:
m = Map(center=(48.1928465, 37.8562077), zoom=5)

In [275]:
#geo_data = GeoData(geo_dataframe=gpd.GeoDataFrame(pd.concat([gdf[0:8], gdf[13:13]])))
geo_data = GeoData(geo_dataframe=ua_territory)

In [276]:
m.add_layer(geo_data)

In [277]:
m

Map(center=[48.1928465, 37.8562077], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [281]:
ua_territory.geometry.area

0     0.167230
1     0.247475
2     0.186683
3     0.426666
4     1.030923
5     1.655796
6     0.060470
7     2.007172
10    3.062121
dtype: float64

In [279]:
ua_territory

,id,name,geometry
0,22397871,Russian occupation in Chernihiv region,"MULTIPOLYGON (((30.65255 51.34204, 30.69649 51..."
1,22397844,Russian occupation in Kyiv region,"MULTIPOLYGON (((29.99079 51.48292, 29.98229 51..."
2,22397831,Russian occupation in Luhansk region,"MULTIPOLYGON (((39.69841 49.04973, 39.72519 49..."
3,22397830,Russian occupation in Kharkiv region,"MULTIPOLYGON (((35.57099 50.45468, 35.45975 50..."
4,22397828,Russian occupation in Sumy region,"MULTIPOLYGON (((34.09059 51.67073, 34.16337 51..."
5,22397843,Russian occupation Kherson region,"MULTIPOLYGON (((34.89464 45.75686, 34.95812 45..."
6,22397706,Zmiyiny,"MULTIPOLYGON (((30.23988 45.14589, 30.33463 45..."
7,21864893,russiainvadedukraine,"MULTIPOLYGON (((39.79403 48.59222, 39.79707 48..."
10,21191074,Occupied Crimea,"MULTIPOLYGON (((34.97680 45.69307, 35.03643 45..."


In [284]:
ua_territory = ua_territory.set_crs('EPSG:4326')

In [285]:
ua_territory.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [287]:
data_copy = ua_territory.copy()
data_copy = data_copy.to_crs({'proj':'cea'})
ua_territory['poly_area'] = data_copy['geometry'].area/ 10**6

In [288]:
ua_territory

,id,name,geometry,poly_area
0,22397871,Russian occupation in Chernihiv region,"MULTIPOLYGON (((30.65255 51.34204, 30.69649 51...",1284.457517
1,22397844,Russian occupation in Kyiv region,"MULTIPOLYGON (((29.99079 51.48292, 29.98229 51...",1920.719180
2,22397831,Russian occupation in Luhansk region,"MULTIPOLYGON (((39.69841 49.04973, 39.72519 49...",1515.047367
3,22397830,Russian occupation in Kharkiv region,"MULTIPOLYGON (((35.57099 50.45468, 35.45975 50...",3382.676242
4,22397828,Russian occupation in Sumy region,"MULTIPOLYGON (((34.09059 51.67073, 34.16337 51...",7988.009032
5,22397843,Russian occupation Kherson region,"MULTIPOLYGON (((34.89464 45.75686, 34.95812 45...",14164.568567
6,22397706,Zmiyiny,"MULTIPOLYGON (((30.23988 45.14589, 30.33463 45...",527.473756
7,21864893,russiainvadedukraine,"MULTIPOLYGON (((39.79403 48.59222, 39.79707 48...",16632.542248
10,21191074,Occupied Crimea,"MULTIPOLYGON (((34.97680 45.69307, 35.03643 45...",26689.319410


In [292]:
ua_territory['poly_area'].sum()

74104.81331960809